In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
n_epochs = 5
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [4]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [5]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [6]:
example_data.shape

torch.Size([1000, 1, 28, 28])

In [7]:
import matplotlib.pyplot as plt

fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
fig

<Figure size 640x480 with 6 Axes>

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [9]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [10]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [11]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), 'C:/Users/lerab/Desktop/Lera/hse 2019-2020/NIRS/virt/results/model.pth')
      torch.save(optimizer.state_dict(), 'C:/Users/lerab/Desktop/Lera/hse 2019-2020/NIRS/virt/results/optimizer.pth')

In [12]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [13]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

c:\users\lerab\desktop\lera\hse201~2\nirs\virt\keras\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
c:\users\lerab\desktop\lera\hse201~2\nirs\virt\keras\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.3004, Accuracy: 751/10000 (7%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.268457
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.268337
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.274000
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.251669
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.236826
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.223578
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.153432
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.096480
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.974480
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.944769
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.839859
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.767560
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.698696
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.683217
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.504576
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.390304
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.288033
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.330192
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.279881
Train Epoch: 1 [12160/6

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.378457
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.457691
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.433024
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.413119
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.262396
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.285272
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.509609
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.346104
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.370518
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.307770
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.289831
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.456398
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.196197
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.394947
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.391156
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.308819
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.326067
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.375526
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.646098
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.329660
